In [3]:
def save_donnees_source_to_s3(prep_path):
    import utils as u
    import os

    if not 'workbookDir' in globals():
        workbookDir = os.getcwd()
    aws_separator = "/"

    # Import S3 connectors librairies
    #from awsPy.aws_authorization import aws_connector
    import importlib.util
    spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
    aws_connector = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(aws_connector)

    #from awsPy.aws_s3 import service_s3
    import importlib.util
    spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
    service_s3 = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(service_s3)

    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')

    # List files to send and compute each destination

    list_files=[]
    for root,dirs,files in os.walk(prep_path):
        for file in files:
              if file.endswith(('.csv')):#####
                    #print(file)
                    list_files.append((file, u.calc_dest(filename=file)))

    # Send files to s3
    for (file_to_upload,destination_in_s3) in list_files:
        # if file exist in S3?
        full_path = os.path.join(prep_path, file_to_upload)
        key = aws_separator.join([destination_in_s3,file_to_upload])
        #print(full_path)
        #print(key)        
        if not s3.key_exist(key):
            s3.upload_file(full_path, destination_in_s3)
    
    return True
   

In [ ]:
prep_path='data/prep2' #stock 2017
save_donnees_source_to_s3(prep_path)

In [4]:
prep_path='data/prep3' #stock 2018
save_donnees_source_to_s3(prep_path)

True

In [ ]:
prep_path='data/prep4' #stock 2019
save_donnees_source_to_s3(prep_path)